In [1]:
from deepface import DeepFace
#liveness
import tensorflow as tf

In [2]:
import cv2, numpy, datetime, pygame, keyboard
import xlwings as xw

pygame 2.5.0 (SDL 2.28.0, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model = 'liveness.model'
model = tf.keras.models.load_model(model)

In [4]:
#xlwings
#initializing workbook and worksheet
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [5]:
#xlwings

worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

In [6]:
s= 2

i = 0

entered = False
al_entered = False

In [ ]:
cap = cv2.VideoCapture(0)

#pygame

pygame.init()

pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background,0,0.8)
screen.blit(background,(0,0))


#day today
t0 = datetime.date.today()
t0 = t0.day

running = True

while running:
    
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year
    
    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"
        
    screen.blit(background,(0,0))
    
    #check 
    
    if day!=t0:
        t0 = day
        
        worksheet = workbook.sheets.add(date)

        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i = 0
        entered = False
        al_entered = False
    
    
    
    
    
    state, frame = cap.read()
    
    if state!=True:
        break
        
    res  = DeepFace.find(frame, db_path='./Database/', enforce_detection=False, model_name='VGG-Face')
    
    if len(res[0]['identity'])>0:
        
        name = res[0]['identity'][0].split('/')[1].split('\\')[1]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        
        xmax = int(xmin + w)
        ymax = int(ymin + h)
        
        #liveness
        
        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32,32))
        img = img.astype('float')/255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = numpy.expand_dims(img, axis=0)
        
        liveness = model.predict(img)
        
        liveness = liveness[0].argmax()
        
        
        

        
        if liveness==1:
            cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
            cv2.rectangle(frame, (xmin, ymin-25),(xmax, ymin),(255,255,255),-1)
            cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)
            
            n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
            n_txtrect = n_txt.get_rect()
            n_txtrect.center =(960-200, 540//2)
            
            d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
            d_txtrect = d_txt.get_rect()
            d_txtrect.center= (960-200, (540//2)+40)
            
            t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
            t_txtrect = d_txt.get_rect()
            t_txtrect.center = (960-200, (540//2)+80)         
            
            
            screen.blit(n_txt, n_txtrect)
            screen.blit(d_txt, d_txtrect)
            screen.blit(t_txt, t_txtrect)
            
            if keyboard.is_pressed('enter'):
                
                if name not in students:
                    
                    worksheet.range(f'A{s}').value = name
                    worksheet.range(f'B{s}').value = date
                    worksheet.range(f'c{s}').value = time
                    students.append(name)
                    s+=1
                    entered =True
                else:
                    al_entered = True
                    
            if i<30 and entered==True:
                e_txt = pygame_font.render('entered', True, pg_txt, pg_bg)
                e_txtrect = e_txt.get_rect()
                e_txtrect.center = (960-200, (540//2)+120)
                screen.blit(e_txt, e_txtrect)
                i+=1

            elif i<30 and al_entered==True:
                ae_txt = pygame_font.render('already entered', True, pg_txt, pg_bg)
                ae_txtrect = ae_txt.get_rect()
                ae_txtrect.center = (960-200, (540//2)+120)
                screen.blit(ae_txt, e_txtrect)    
                i+=1

            else:
                entered=False
                al_entered = False
                i = 0

    
    
    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
    image = pygame.transform.rotozoom(image, 0 , 0.8)
    #round corners--
    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img, (255,255,255), (0,0,*size), border_radius=10)
    
    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    #round corner ---
    
    screen.blit(image, (20,80))
    
    
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    #cv2.imshow('attendance', frame)
    
    #c = cv2.waitKey(1)
    
    #if c == ord('q'):
        #break
        
cap.release()
pygame.quit()
#cv2.destroyAllWindows()


There are  9  representations found in  representations_vgg_face.pkl
find function lasts  1.3765513896942139  seconds
1/1 [==============================] - 0s 80ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3050196170806885  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.30482912063598633  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16868352890014648  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1750023365020752  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.304502010345459  seconds
1/1 [==========================

find function lasts  0.2995154857635498  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16663455963134766  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17006683349609375  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16702628135681152  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16693472862243652  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.30116748809814453  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  repr

find function lasts  0.16352415084838867  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.30455946922302246  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.32625246047973633  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17020511627197266  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3103957176208496  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3027324676513672  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  repre

find function lasts  0.16692829132080078  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16556477546691895  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1659555435180664  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16275620460510254  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2999851703643799  seconds
1/1 [==============================] - 0s 11ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1662147045135498  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  repres

1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1581432819366455  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1555490493774414  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.15445590019226074  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16756629943847656  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1526503562927246  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1638

...

find function lasts  0.18538880348205566  seconds
1/1 [==============================] - 0s 15ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1843397617340088  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18799352645874023  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18326759338378906  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18192243576049805  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1871805191040039  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repre

find function lasts  0.18716883659362793  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1879291534423828  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18351459503173828  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18503165245056152  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18235516548156738  seconds
1/1 [==============================] - 0s 15ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18480491638183594  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repr

find function lasts  0.1870250701904297  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1865220069885254  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18695688247680664  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18448758125305176  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1862494945526123  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19168663024902344  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repres

find function lasts  0.18854427337646484  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18583369255065918  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18427395820617676  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18417954444885254  seconds
1/1 [==============================] - 0s 15ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18796348571777344  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1851356029510498  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repr

find function lasts  0.19412684440612793  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1967165470123291  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19788026809692383  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1928846836090088  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19351840019226074  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19916367530822754  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repre